In [1]:
! pip install git+https://github.com/RalfKellner/fintextanalytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/RalfKellner/fintextanalytics to /tmp/pip-req-build-0ubfoc2b
  Running command git clone --filter=blob:none --quiet https://github.com/RalfKellner/fintextanalytics /tmp/pip-req-build-0ubfoc2b
  Resolved https://github.com/RalfKellner/fintextanalytics to commit ec7aee1e5c635f0240243353b6c7666b4f5570cc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.0/594.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 74.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 13

In [1]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load data, preprocess and convert to document embeddings

In [2]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd

import pandas as pd
data = pd.read_csv('/content/drive/My Drive/KalteneckerNew.csv')
data.head()

#rename the colume
data = data.rename(columns={'Unnamed: 0': 'item'})
print(data.head())

#test to grap the diffrent companies
data.loc[data['company'] != 'HOME DEPOT, INC.']

## Company 1
# HOME DEPOT, INC.
company1_data = data[data['company'] == 'HOME DEPOT, INC.']
df1 = company1_data

## Company 2
# CHEVRON CORP
company2_data = data[data['company'] == 'CHEVRON CORP']
df2 = company2_data



     item                                               text  \
0  item1a  >Item 1A.  Risk Factors. Our business, results...   
1  item1b  >Item 1B.  Unresolved Staff Comments. Not appl...   
2   item3  >Item 3.  Legal Proceedings. The Company is pa...   
3   item7  >Item 7.  Management’s Discussion and Analysis...   
4  item7a  >Item 7A.  Quantitative and Qualitative Disclo...   

                    accDate           company  
0  2023-03-15T16:22:15.000Z  HOME DEPOT, INC.  
1  2023-03-15T16:22:15.000Z  HOME DEPOT, INC.  
2  2023-03-15T16:22:15.000Z  HOME DEPOT, INC.  
3  2023-03-15T16:22:15.000Z  HOME DEPOT, INC.  
4  2023-03-15T16:22:15.000Z  HOME DEPOT, INC.  


In [ ]:
from fintextanalytics.utils import report2sentences

# use the report2sentences function to iterate through each item, split into sentences and preprocess text
initialized = False
for idx, row in df.iterrows():
    if initialized:
        report_sentences_prep_tmp = report2sentences(row['text'], idx_start=report_sentences_preprocessed.index[-1] + 1, item = row['item'], accDate = row['accDate'], company = row['company'])
        report_sentences_preprocessed = pd.concat((report_sentences_preprocessed, report_sentences_prep_tmp), axis = 0)
    else:
        report_sentences_prep_tmp = report2sentences(row['text'], item = row['item'], accDate = row['accDate'], company = row['company'])
        report_sentences_preprocessed = report_sentences_prep_tmp.copy()
        initialized = True

# a mapping between docid (sentences which are used for search) and raw id
docid2idx = dict()
for i, idx in enumerate(report_sentences_preprocessed[report_sentences_preprocessed.use_doc].index):
    docid2idx[i] = idx


In [ ]:
from fintextanalytics.embeddingmodels import EmbeddingAnalyzer

embed_analyzer = EmbeddingAnalyzer()

# get the preprocessed text from all documents we want to use
docs = report_sentences_preprocessed[report_sentences_preprocessed.use_doc].prep_text
# use the embed_analzyer doc2vec model to infer document vectors for each document
doc_vecs = [embed_analyzer.doc_model.infer_vector(doc.split(',')) for doc in docs]


In sum 49 words from the default word lists could not be found in the Word2Vec vocabulary. These words are deleted from the word list:
{'cobc', 'transgender', 'householding', 'webpage', 'nondiscrimination', 'perquisites', 'bisexual', 'ethnically', 'marriage', 'ethnicities', 'discriminated', 'stepparents', 'humanity', 'lgbt', 'lesbians', 'siblings', 'educates', 'spousal', 'ethnicity', 'ungc', 'donates', 'vacancyappreciation', 'donating', 'stepchildren', 'grandchildren', 'ilo', 'lobbyist', 'nephews', 'ruggie', 'ballots', 'ethic', 'gays', 'lesbian', 'minorities', 'eicc', 'interlocks', 'motivates', 'proponent', 'homosexual', 'gay', 'nieces', 'vestsballot', 'grandparent', 'eeo', 'childbirth', 'genders', 'honesty', 'grandparents', 'parachutes'}
Vectors seem not have unit length...the vectors have been l2-normalized.


# Find documents close to certain words of interest and analyze

In [ ]:
# the first data frame contains a word and its closest documents, the second includes corresponding cosine similarities
topn_docs, topn_doc_scores = embed_analyzer.find_close_docs_to_words(doc_vecs, wordlist = 'ewords')

unique_topn_docs = topn_docs.loc[:, 0].unique().tolist()
#unique_topn_docs.sort()
original_topn_docs_ids = [docid2idx[id] for id in unique_topn_docs]

for idx, row in report_sentences_preprocessed.loc[original_topn_docs_ids, :].iterrows():
  print('-'*50)
  print(row['raw_text'])
  print('-'*50)

Vectors seem not have unit length...the vectors have been l2-normalized.
--------------------------------------------------
Component defects could make the Company’s products unsafe and create a risk of environmental or property damage and personal injury.
--------------------------------------------------
--------------------------------------------------
The Company’s global operations are subject to complex and changing laws and regulations on subjects, including antitrust; privacy, data security and data localization; consumer protection; advertising, sales, billing and e-commerce; product liability; intellectual property ownership and infringement; digital platforms; Internet, telecommunications, and mobile communications; media, television, film and digital content; availability of third-party software applications and services; labor and employment; anticorruption; import, export and trade; foreign exchange controls and cash repatriation restrictions; anti–money laundering; for

# Include a similarity threshold filter

In [ ]:
mask = topn_docs[topn_doc_scores > 0.35]

mask_ids = []
for idx, row in mask.iterrows():
  tmp = row[row.notnull()].tolist()
  if len(tmp) > 0:
    mask_ids = mask_ids + tmp

mask_ids = list(set(mask_ids))
mask_ids = [int(id) for id in mask_ids]
original_mask_ids = [docid2idx[id] for id in mask_ids]

for idx, row in report_sentences_preprocessed.loc[original_mask_ids, :].iterrows():
  print('-'*50)
  print(row['raw_text'])
  print('-'*50)

--------------------------------------------------
Global climate change could result in certain types of natural disasters occurring more frequently or with more intense effects.
--------------------------------------------------
--------------------------------------------------
Many governments, regulators, investors, employees, customers and other stakeholders are increasingly focused on environmental, social and governance considerations relating to businesses, including climate change and greenhouse gas emissions, human and civil rights, and diversity, equity and inclusion.
--------------------------------------------------
--------------------------------------------------
Global climate change could result in certain types of natural disasters occurring more frequently or with more intense effects.
--------------------------------------------------
--------------------------------------------------
The Company cannot guarantee that it will achieve its announced environmental, s

In [ ]:
report_sentences_preprocessed.loc[original_mask_ids, :]

,raw_text,prep_text,use_doc,item,accDate,company
4621,Global climate change could result in certain ...,"global,climate,change,could,result,in,certain,...",True,item1a,2019-10-30T18:12:36.000Z,Apple
6119,"Many governments, regulators, investors, emplo...","many,governments,regulators,investors,employee...",True,item1a,2022-10-27T18:01:14.000Z,Apple
5136,Global climate change could result in certain ...,"global,climate,change,could,result,in,certain,...",True,item1a,2020-10-29T18:06:25.000Z,Apple
6122,The Company cannot guarantee that it will achi...,"the,company,cannot,guarantee,that,it,will,achi...",True,item1a,2022-10-27T18:01:14.000Z,Apple
5916,Global climate change is resulting in certain ...,"global,climate,change,is,resulting,in,certain,...",True,item1a,2022-10-27T18:01:14.000Z,Apple
5440,Global climate change is resulting in certain ...,"global,climate,change,is,resulting,in,certain,...",True,item1a,2021-10-28T18:04:28.000Z,Apple


In [ ]:
for txt in report_sentences_preprocessed.loc[6117:6124].raw_text:
  print(txt)

As a result, the Company’s products could be delayed or prohibited from importation, either of which could materially adversely affect the Company’s business, reputation, results of operations and financial condition.
Expectations relating to environmental, social and governance considerations expose the Company to potential liabilities, increased costs, reputational harm, and other adverse effects on the Company’s business.
Many governments, regulators, investors, employees, customers and other stakeholders are increasingly focused on environmental, social and governance considerations relating to businesses, including climate change and greenhouse gas emissions, human and civil rights, and diversity, equity and inclusion.
In addition, the Company makes statements about its environmental, social and governance goals and initiatives through its environmental, social and governance report, its other non-financial reports, information provided on its website, press statements and other c

In [ ]:
embed_analyzer.word_list['esgwords']

['clean',
 'environmental',
 'epa',
 'sustainability',
 'climate',
 'warming',
 'biofuels',
 'biofuel',
 'green',
 'renewable',
 'solar',
 'stewardship',
 'wind',
 'emission',
 'emissions',
 'ghg',
 'ghgs',
 'greenhouse',
 'atmosphere',
 'emit',
 'agriculture',
 'deforestation',
 'pesticide',
 'pesticides',
 'wetlands',
 'zoning',
 'biodiversity',
 'species',
 'wilderness',
 'wildlife',
 'freshwater',
 'groundwater',
 'water',
 'air',
 'carbon',
 'nitrogen',
 'pollution',
 'superfund',
 'hazardous',
 'pollutants',
 'printing',
 'recycling',
 'toxic',
 'waste',
 'wastes',
 'weee',
 'recycle',
 'citizen',
 'citizens',
 'csr',
 'disabilities',
 'disability',
 'disabled',
 'human',
 'nations',
 'social',
 'un',
 'veteran',
 'veterans',
 'vulnerable',
 'children',
 'epidemic',
 'health',
 'healthy',
 'ill',
 'illness',
 'pandemic',
 'drug',
 'medicaid',
 'medicare',
 'medicine',
 'medicines',
 'hiv',
 'alcohol',
 'drinking',
 'bugs',
 'conformance',
 'defects',
 'fda',
 'inspection',
 'insp

In [ ]:
e_word_list = embed_analyzer.word_list['ewords']

In [ ]:
topn_words, topn_word_scores = embed_analyzer.find_close_words_to_docs(doc_vecs)

Vectors seem not have unit length...the vectors have been l2-normalized.


In [ ]:
for idx in topn_words.index:
  if topn_words.loc[idx, 0] in e_word_list:
    print('-' * 50)
    print(topn_words.loc[idx, 0])
    print('-' * 25)
    print(topn_word_scores.loc[idx, 0])
    print('-' * 25)
    print(report_sentences_preprocessed.loc[docid2idx[idx]].raw_text)
    print('-' * 50)

--------------------------------------------------
biofuels
-------------------------
0.18371442
-------------------------
In contrast, many of the Companys competitors seek to compete primarily through aggressive pricing and very low cost structures, and emulating the Companys products and
infringing its intellectual property.
--------------------------------------------------
--------------------------------------------------
biofuels
-------------------------
0.21722157
-------------------------
To remain
competitive and stimulate customer demand, the Company must successfully manage frequent product introductions and transitions.
--------------------------------------------------
--------------------------------------------------
biofuels
-------------------------
0.17083487
-------------------------
Due to the highly volatile and competitive nature of the industries in which the Company competes, the Company must continually introduce new products, services and technologies, enh

In [ ]:
embed_analyzer.fit_topic_model(doc_vecs)

Vectors seem not have unit length...the vectors have been l2-normalized.
Starting to train the dimensionality reduction model...
Umap model for dimensionality reduction has been trained!

Starting to train the clustering model...
HDBSCAN model for clustering has been trained!

Creating topic vectors...
Creating topic vectors finished! 17 topics have been found.
Creating a reduced topic model...
The number of topics has been reduced from 17 to 14.
Topic reduction is finished, the number of reduced topics is 14


In [ ]:
embed_analyzer.topic_words

array([['innovate', 'scalability', 'software', 'functionality',
        'striving', 'deploy', 'adapt', 'embed', 'engineer', 'enhance',
        'datav', 'roadmap', 'redundancy', 'technologies', 'enhancements',
        'introduce', 'technologically', 'sharpen', 'posture', 'agility',
        'competitively', 'mainframe', 'cxm', 'vulnerabilities',
        'integrations'],
       ['usernames', 'fraudulently', 'passwords', 'unwittingly',
        'sensitive', 'spoof', 'cpni', 'privileged', 'steal', 'insecure',
        'spoofing', 'disclosing', 'directory', 'subscribes', 'entrusted',
        'stealing', 'email', 'login', 'induce', 'personally',
        'unintentional', 'erroneous', 'transmit', 'access', 'biller'],
       ['vsoe', 'undelivered', 'esps', 'deliverables', 'unspecified',
        'ssp', 'bug', 'fixes', 'deliverable', 'ssps', 'ratably', 'besp',
        'standalone', 'badges', 'element', 'bundled', 'esp', 'pcs',
        'software', 'separable', 'siri', 'nonsoftware', 'spok', 'tpe',
  

In [ ]:
from fintextanalytics.utils import write_pickle

write_pickle('/content/drive/MyDrive/Colab Notebooks/AML/appleEmbeddingModel', embed_analyzer)

In [ ]:
from fintextanalytics.utils import read_pickle

scnd_embed_analyzer = read_pickle('/content/drive/MyDrive/Colab Notebooks/AML/appleEmbeddingModel')

In [ ]:
scnd_embed_analyzer.topic_words

array([['innovate', 'scalability', 'software', 'functionality',
        'striving', 'deploy', 'adapt', 'embed', 'engineer', 'enhance',
        'datav', 'roadmap', 'redundancy', 'technologies', 'enhancements',
        'introduce', 'technologically', 'sharpen', 'posture', 'agility',
        'competitively', 'mainframe', 'cxm', 'vulnerabilities',
        'integrations'],
       ['usernames', 'fraudulently', 'passwords', 'unwittingly',
        'sensitive', 'spoof', 'cpni', 'privileged', 'steal', 'insecure',
        'spoofing', 'disclosing', 'directory', 'subscribes', 'entrusted',
        'stealing', 'email', 'login', 'induce', 'personally',
        'unintentional', 'erroneous', 'transmit', 'access', 'biller'],
       ['vsoe', 'undelivered', 'esps', 'deliverables', 'unspecified',
        'ssp', 'bug', 'fixes', 'deliverable', 'ssps', 'ratably', 'besp',
        'standalone', 'badges', 'element', 'bundled', 'esp', 'pcs',
        'software', 'separable', 'siri', 'nonsoftware', 'spok', 'tpe',
  

In [ ]:
scnd_embed_analyzer.word_list.keys()

dict_keys(['ewords', 'swords', 'gwords', 'esgwords'])